In [21]:
# FOLDER PATHS
# period 1 exams
I_mon1 = "data/enrolment-data-cleaned-2024-period-I/MondayOct14-900-1200"
I_mon2 = "data/enrolment-data-cleaned-2024-period-I/MondayOct14-1300-1600"
I_tue = "data/enrolment-data-cleaned-2024-period-I/TuesdayOct15"
I_wed = "data/enrolment-data-cleaned-2024-period-I/WednesdayOct16"
I_thu = "data/enrolment-data-cleaned-2024-period-I/ThursdayOct17"

# period 2 exams
II_mon1 = "data/enrolment-data-cleaned-2024-period-II/Monday-2-12_cleaned"
II_mon2 = "data/enrolment-data-cleaned-2024-period-II/Monday-9-12_cleaned"
II_tue1 = "data/enrolment-data-cleaned-2024-period-II/Tuesday-3-12-900_cleaned"
II_tue2 = "data/enrolment-data-cleaned-2024-period-II/Tuesday-3-12-1630_cleaned"
II_tue3 = "data/enrolment-data-cleaned-2024-period-II/Tuesday-10-12_cleaned"
II_wed = "data/enrolment-data-cleaned-2024-period-II/Wednesday-11-12_cleaned"
II_thu1 = "data/enrolment-data-cleaned-2024-period-II/Thursday-5-12_cleaned"
II_thu2 = "data/enrolment-data-cleaned-2024-period-II/Thursday-12-12-900_cleaned"
II_thu3 = "data/enrolment-data-cleaned-2024-period-II/Thursday-12-12-1630_cleaned"


In [22]:
# Imports
import os
from collections import defaultdict

import pandas as pd
import numpy as np

import pyoptinterface as poi
from pyoptinterface import gurobi

# Constants
EXAM_DATA_FOLDER = "data/registration-2-12-DRAFT/"
SUPERVISOR_COST = 16  # average hourly cost of hiring one exam supervisor
EXAM_DURATION_HOURS = 4  # 3-hour-exams + 1 hour buffer for reservations/working hours
# E.g. these work:
#RESERVED_HALLS = ["A (Aalto)", "A1", "A2", "A3", "AS 1", "AS 2", "B", "C", "D", "E", "Jeti (A208d)", "M1", "M2", "M3", "T1", "T2", "TU 1", "TU 2", "U1", "U2", "U3", "U4", "U5", "U6", "Y313", "Y405"]
#RESERVED_HALLS = ["A (Aalto)", "AS 2", "B", "C", "D", "E", "Jeti (A208d)", "T1","TU 1", "TU 2", "U1", "U2"]
# But this does not:
RESERVED_HALLS = ['A (Aalto)', 'AS 2', 'B', 'C', 'D', 'E', 'Jetti (A208d)', 'T1', 'TU 2', 'U2']
# [cost, spacing, splits]
W = [1,3,1]

In [23]:
def load_course_participation(folder_path):
    course_counts = defaultdict(int)

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            course_code = filename[:5]
            file_path = os.path.join(EXAM_DATA_FOLDER, filename)
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            confirmed_count = (df['ENROLMENT STATUS'] == 'Confirmed').sum()
            course_counts[course_code] += confirmed_count

    return pd.DataFrame(list(course_counts.items()), columns=['Course Code', 'Confirmed Count'])

# def load_course_participation(folder_path):
#     course_counts = defaultdict(int)
#
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.csv'):
#             course_code = filename[:5]
#             file_path = os.path.join(folder_path, filename)
#             df = pd.read_csv(file_path)
#             confirmed_count = (df['ENROLMENT STATUS'] == 'Confirmed').sum()
#             course_counts[course_code] += confirmed_count
#
#     return pd.DataFrame(list(course_counts.items()), columns=['Course Code', 'Confirmed Count'])


def process_hall_sheet(df):
    halls_summary = []

    for hall in df['Sali'].unique():
        hall_data = df[df['Sali'] == hall]

        odd_rows = hall_data[hall_data['Rivinro salissa'] % 2 == 1]
        even_rows = hall_data[hall_data['Rivinro salissa'] % 2 == 0]
        total_odd_seats = odd_rows['Tenttipaikkoja rivillä'].sum()
        total_even_seats = even_rows['Tenttipaikkoja rivillä'].sum()

        halls_summary.append({
            'Hall': hall,
            'Odd_row_seats': total_odd_seats,
            'Even_row_seats': total_even_seats
        })

    return pd.DataFrame(halls_summary)

def load_hall_data(file_path):
    """Load and process all hall layout data."""
    sheet1 = pd.read_excel(file_path, sheet_name='Exam_halls_1_seat').sort_values('Sali').reset_index(drop=True)
    sheet2 = pd.read_excel(file_path, sheet_name='Exam_halls_2_seat').sort_values('Sali').reset_index(drop=True)

    sheet1['Sali'] = sheet1['Sali'].str.replace('U2 (Mellin)', 'U2')
    sheet2['Sali'] = sheet2['Sali'].str.replace('U2 (Mellin)', 'U2')

    sheet1['Sali'] = sheet1['Sali'].str.replace('Jetti (A208d)', 'Jeti (A208d)')
    sheet2['Sali'] = sheet2['Sali'].str.replace('Jetti (A208d)', 'Jeti (A208d)')

    summary_1_seat = process_hall_sheet(sheet1)
    summary_2_seat = process_hall_sheet(sheet2)

    return summary_1_seat.merge(summary_2_seat, on='Hall', suffixes=('_1', '_2'))

def normalize_hall_names(df, col='Sali'):
    # literal replace (parentheses are literal), do all replacements in one place
    df[col] = df[col].str.replace('U2 (Mellin)', 'U2', regex=False)
    df[col] = df[col].str.replace('Jetti (A208d)', 'Jeti (A208d)', regex=False)
    return df

def load_row_data(file_path):
    df_1 = pd.read_excel(file_path, sheet_name='Exam_halls_1_seat')
    df_2 = pd.read_excel(file_path, sheet_name='Exam_halls_2_seat')

    df_1 = normalize_hall_names(pd.read_excel(file_path, sheet_name='Exam_halls_1_seat'))
    df_2 = normalize_hall_names(pd.read_excel(file_path, sheet_name='Exam_halls_2_seat'))

    merged = pd.merge(
        df_1[['Sali', 'Rivinro salissa', 'Tenttipaikkoja rivillä']],
        df_2[['Sali', 'Rivinro salissa', 'Tenttipaikkoja rivillä']],
        on=['Sali', 'Rivinro salissa'],
        suffixes=('_1', '_2')
    )

    # Assign hall index alphabetically
    hall_order = {hall: idx for idx, hall in enumerate(sorted(merged['Sali'].unique()))}
    merged['hall_index'] = merged['Sali'].map(hall_order)

    # Row index within each hall (0-based index)
    merged['row_index'] = merged.groupby('Sali').cumcount()

    # Add total number of rows per hall
    row_counts = merged.groupby('Sali')['Rivinro salissa'].nunique().reset_index()
    row_counts.rename(columns={'Rivinro salissa': 'n_rows'}, inplace=True)
    merged = merged.merge(row_counts, on='Sali')

    # Final DataFrame
    result = merged[['hall_index', 'Sali', 'row_index', 'n_rows', 'Rivinro salissa', 'Tenttipaikkoja rivillä_1', 'Tenttipaikkoja rivillä_2']]
    result.columns = ['hall_index', 'hall_name', 'row_index', 'n_rows', 'row_number', 'seats_1', 'seats_2']

    return result



def set_reserved_halls(row_data, price_data, reserved_halls):
    filtered_rows = row_data[row_data['hall_name'].isin(reserved_halls)].copy()
    filtered_prices = price_data[price_data['Sali'].isin(reserved_halls)].copy()

    filtered_rows = filtered_rows.sort_values("hall_name").reset_index(drop=True)
    filtered_prices = filtered_prices.sort_values("Sali").reset_index(drop=True)

    hall_index_map = {name: idx for idx, name in enumerate(filtered_rows['hall_name'].unique())}

    filtered_rows['hall_index'] = filtered_rows['hall_name'].map(hall_index_map)
    filtered_prices['hall_index'] = filtered_prices['Sali'].map(hall_index_map)

    return filtered_rows, filtered_prices

def combine_courses(course_data, combined_courses, new_course_code):
    rows_to_combine = course_data[course_data['Course Code'].isin(combined_courses)]

    total_count = rows_to_combine['Confirmed Count'].sum()

    updated_data = course_data[~course_data['Course Code'].isin(combined_courses)].copy()

    new_row = pd.DataFrame([{
         'Course Code': new_course_code,
         'Confirmed Count': total_count
    }])

    return pd.concat([updated_data, new_row], ignore_index=True).sort_values('Course Code').reset_index(drop=True)


In [24]:
def setup_optimization_model(courses, rows, prices):
    """Set up and solve the optimization problem."""
    model = gurobi.Model()

    # Prepare data
    sup_per_hall = prices["Valvojia salissa"].tolist()
    sup_cost = [c * SUPERVISOR_COST * EXAM_DURATION_HOURS for c in sup_per_hall]
    size = courses["Confirmed Count"].tolist()

    # Calculating total cost
    hall_prices = [n * EXAM_DURATION_HOURS for n in prices["Tilavuokra/h"].tolist()]
    cost = np.add(hall_prices, sup_cost)

    # Indexing
    course_codes = courses["Course Code"].tolist()
    hall_names = rows["hall_name"].unique().tolist()
    n_courses = len(courses)
    n_halls = len(rows['hall_name'].unique())
    n_students = sum(size)

    n_rows = [rows[rows['hall_index'] == h]['row_index'].nunique() for h in range(n_halls)]
    modes = [1,2]

    # Hall capacities
    cap = {}
    for m in modes:
        for h in range(n_halls):
            for r in range(n_rows[h]):
                v = rows[(rows['hall_index'] == h) & (rows['row_index'] == r)][f'seats_{m}'].values
                cap[m, h, r] = int(v[0]) if len(v) > 0 else 0

    # Decision variables
    x = {} # Course assignments to rows
    y = {} # Hall usage
    z = {} # Hall usage by course

    # 1 if course c assigned to row r in hall h
    for m in modes:
        for c in range(n_courses):
            for h in range(n_halls):
                    for r in range(n_rows[h]):
                        x[m,c,h,r] = model.add_variable(name=f"x_{m}_{c}_{h}_{r}", domain=poi.VariableDomain.Binary)

    # 1 if hall h used
    for m in modes:
        for h in range(n_halls):
            y[m, h] = model.add_variable(name=f"y_{m}_{h}", domain=poi.VariableDomain.Binary)

    # 1 if course c has students in hall h
    for c in range(n_courses):
        for h in range(n_halls):
            z[c,h] = model.add_variable(name=f"z_{c}_{h}", domain=poi.VariableDomain.Binary)


    # Adding constrains

    # The capacities of the rows assigned to course c add up at least to the size of course c
    for c in range(n_courses):
        expr = poi.ExprBuilder()
        for h in range(n_halls):
            for r in range(n_rows[h]):
                for m in modes:
                    expr += x[m,c,h,r]*cap[m,h,r]
        expr -= size[c]
        model.add_linear_constraint(expr, poi.ConstraintSense.GreaterEqual, 0)

    # only one course can be assigned to a row
    for h in range(n_halls):
        for r in range(n_rows[h]):
            for m in modes:
                expr = poi.ExprBuilder()
                for c in range(n_courses):
                    expr += x[m,c,h,r]
                expr -= y[m,h]
                model.add_linear_constraint(expr, poi.ConstraintSense.LessEqual, 0)

    # if student from course c is in hall h row r then hall h is used by course c
    for c in range(n_courses):
        for h in range(n_halls):
            for r in range(n_rows[h]):
                for m in modes:
                    expr = x[m,c,h,r] - z[c,h]
                    model.add_linear_constraint(expr, poi.ConstraintSense.LessEqual, 0)

    # bound z, z[c,h] = 1 if only if at least one of x[m,c,h,r] = 1
    for c in range(n_courses):
        for h in range(n_halls):
            expr = poi.ExprBuilder()
            for m in modes:
                for r in range(n_rows[h]):
                    expr += x[m, c, h, r]
            expr -= z[c, h]
            model.add_linear_constraint(expr, poi.ConstraintSense.GreaterEqual, 0)

    # prevent same-course students in adjacent rows
    for c in range(n_courses):
        for h in range(n_halls):
            for r in range(n_rows[h]-1):
                for m in modes:
                    expr = x[m,c,h,r] + x[m,c,h,r+1]
                    model.add_linear_constraint(expr, poi.ConstraintSense.LessEqual, 1)

    # only one seating type in use per hall
    for h in range(n_halls):
        expr = poi.ExprBuilder()
        for m in modes:
            expr += y[m,h]
        model.add_linear_constraint(expr, poi.ConstraintSense.LessEqual, 1)

    # setting multi-objective function
    f1 = poi.ExprBuilder()
    for h in range(n_halls):
        for m in modes:
            f1 += y[m,h] * cost[h]

    f2 = poi.ExprBuilder()
    for c in range(n_courses):
        for h in range(n_halls):
            for r in range(n_rows[h]):
                f2 += cap[2,h,r] * x[2,c,h,r]

    f3 = poi.ExprBuilder()
    for c in range(n_courses):
        for h in range(n_halls):
            f3 += z[c,h] - 1

    model.set_objective(W[0]*f1 - W[1]*f2 + W[2]*f3, poi.ObjectiveSense.Minimize)

    return model, {
        'x': x, 'y': y, 'z': z,
        'course_codes': course_codes,
        'hall_names': hall_names,
        'n_courses': n_courses,
        'n_halls': n_halls,
        'cap': cap,
        'cost': cost,
        'hall_prices': hall_prices,
        'sup_cost': sup_cost,
        'participants': size
    }

In [25]:
def print_results(model, context):

    x = context['x']
    y = context['y']
    z = context['z']
    cap = context['cap']
    course_codes = context['course_codes']
    hall_names = context['hall_names']
    n_courses = context['n_courses']
    n_halls = context['n_halls']
    cost = context['cost']
    participants = context['participants']
    modes = [1, 2]  # seating options

    actual_cost = 0
    for h in range(n_halls):
        for m in modes:
            if model.get_value(y[m, h]) > 0.9:
                actual_cost += cost[h]

    print(f"\nOptimized hall reservation cost: €{actual_cost}")

    objval = model.get_model_attribute(poi.ModelAttribute.ObjectiveValue)
    print(f"Objective value (with weights): {objval}")

    # Per-course breakdown
    for c in range(n_courses):

        split_count = sum(1 for h in range(n_halls) if model.get_value(z[c, h]) > 0.9)

        print(f"Course {course_codes[c]} (participants {participants[c]}, splits {split_count - 1}):")
        for h in range(n_halls):
            for m in modes:
                rows_used = []
                for r in range(len([k for k in cap if k[0] == m and k[1] == h])):
                    if model.get_value(x[m, c, h, r]) > 0.9:
                        rows_used.append((r, cap[m, h, r]))
                if rows_used:
                    print(f"  → {hall_names[h]} (mode {m}-seat):")
                    for r_idx, seats in rows_used:
                        print(f"     - Row {r_idx+1} with {seats} seats")
        print()

    print("Hall usage summary:")
    for h in range(n_halls):
        mode_used = None
        for m in modes:
            if model.get_value(y[m, h]) > 0.9:
                mode_used = m
                break
        if mode_used is not None:
            total_seats = sum(cap[mode_used, h, r] for (m2, h2, r) in cap if m2 == mode_used and h2 == h)
            assigned_seats = sum(model.get_value(x[mode_used, c, h, r]) * cap[mode_used, h, r]
                                 for c in range(n_courses)
                                 for r in range(len([k for k in cap if k[0] == mode_used and k[1] == h])))
            usage_pct = 100 * assigned_seats / total_seats if total_seats else 0
            print(f"- {hall_names[h]} used (mode {mode_used}-seat) | "
                  f"Cost: €{cost[h]:.2f} | Usage: {assigned_seats:.0f}/{total_seats} ({usage_pct:.1f}%)")

def print_short(model, context):

    x = context['x']
    y = context['y']
    z = context['z']
    cap = context['cap']
    course_codes = context['course_codes']
    hall_names = context['hall_names']
    n_courses = context['n_courses']
    n_halls = context['n_halls']
    cost = context['cost']
    participants = context['participants']
    modes = [1, 2]  # seating options

    actual_cost = 0
    for h in range(n_halls):
        for m in modes:
            if model.get_value(y[m, h]) > 0.9:
                actual_cost += cost[h]

    print(f"\nActual hall reservation cost: €{actual_cost}\n")

    # Per-course breakdown
    for c in range(n_courses):

        split_count = sum(1 for h in range(n_halls) if model.get_value(z[c, h]) > 0.9)

        print(f"Course {course_codes[c]} (participants {participants[c]}, splits {split_count - 1}):")
        for h in range(n_halls):
            for m in modes:
                rows_used = []
                for r in range(len([k for k in cap if k[0] == m and k[1] == h])):
                    if model.get_value(x[m, c, h, r]) > 0.9:
                        rows_used.append((r, cap[m, h, r]))
                if rows_used:
                    print(f"  → {hall_names[h]}, rows: ", end='')
                    for r_idx, seats in rows_used:
                        print(r_idx+1, end=' ')
                    print()
        print()



In [26]:
course_data = load_course_participation(EXAM_DATA_FOLDER)
#hall_data = load_hall_data("data/Exam_halls_and_example.xlsx")
row_data = load_row_data("data/Exam_halls_and_example.xlsx")
#price_data = pd.read_excel("data/Exam_halls_and_example.xlsx", sheet_name='Hall_prices').sort_values('Sali').reset_index(drop=True)
price_data = pd.read_excel("data/Exam_halls_and_example.xlsx", sheet_name='Hall_prices')
price_data = normalize_hall_names(price_data, col='Sali') \
    .sort_values('Sali') \
    .reset_index(drop=True)

# This can be used to set the halls that have been reserved
row_data, price_data = set_reserved_halls(row_data, price_data, RESERVED_HALLS)

# This can be used to combine courses into one
#course_data = combine_courses(course_data, ['A0501','A0502','A0503','A0504'], 'A05XX')

# Set up and solve optimization
model, context = setup_optimization_model(course_data, row_data, price_data)
model.optimize()

# Print results
print_results(model, context)
print_short(model, context)


Optimized hall reservation cost: €6127.44
Objective value (with weights): 4067.4399999999996
Course A0001 (participants 126, splits 2):
  → B (mode 2-seat):
     - Row 1 with 5 seats
     - Row 3 with 7 seats
     - Row 5 with 8 seats
     - Row 7 with 8 seats
     - Row 9 with 8 seats
     - Row 11 with 11 seats
     - Row 13 with 9 seats
  → D (mode 2-seat):
     - Row 1 with 6 seats
     - Row 3 with 6 seats
     - Row 5 with 6 seats
     - Row 7 with 6 seats
     - Row 9 with 6 seats
     - Row 11 with 6 seats
     - Row 13 with 6 seats
     - Row 15 with 2 seats
  → T1 (mode 1-seat):
     - Row 4 with 13 seats
     - Row 8 with 13 seats

Course A0003 (participants 587, splits 8):
  → A (Aalto) (mode 2-seat):
     - Row 1 with 8 seats
     - Row 3 with 9 seats
     - Row 5 with 11 seats
     - Row 7 with 12 seats
     - Row 9 with 13 seats
     - Row 11 with 14 seats
     - Row 13 with 10 seats
     - Row 15 with 10 seats
  → AS 2 (mode 2-seat):
     - Row 2 with 7 seats
     - Ro